In [1]:
import csv
import pandas as pd
import string
# import requests
from xml.dom import minidom
import nltk
from bs4 import BeautifulSoup

In [2]:
import xml.etree.ElementTree as ET

def xml2df(path):
    tree = ET.parse(path)
    root = tree.getroot()
    data=[]
    for child in root:
        #print(child.attrib)
        #df = pd.Series(child.attrib).to_frame()
        data.append(child.attrib)
    df = pd.DataFrame(data)
    return df

In [ ]:
# Posts.xml 
# PostTypeId = 1 Qeustion, Id: QuestionId, Tags: Category, OwnerUserId: AskerId(UserId), Tags: Category,
# PostTypeId = 2 Answer, Id: AnswerId, ParentId： QuestionId, Score: Helpfulness, CreationDate: Timestamp, 
# OwnerUserId: AnswererId(UserId)
# Users.xml Reputation: Reputation, DsiplayName: Username, Id: UserId
# Tags.xml TagName: Category

In [54]:
from pathlib import Path
abs_data_dir = Path.cwd() / 'csv_files/stackexchange/content'
dataset_dir = Path.cwd() / 'csv_files/stackexchange'


def create_data(category):
    df_post= xml2df(abs_data_dir / 'Posts.xml')
    df_user= xml2df(abs_data_dir / 'Users.xml')
    
    print("Posts: ",df_post.shape)
    df_post = df_post[['OwnerUserId', 'Body', 'PostTypeId', 'CreationDate', 'Score']]
    df_post = df_post[df_post['PostTypeId'].apply(lambda x: x == '2')]
    df_post.rename(columns={'OwnerUserId': 'UserId'}, inplace=True)

    print("Users: ", df_user.shape)
    df_user = df_user[['Id', 'DisplayName', 'Reputation']]
    df_user.rename(columns={'Id': 'UserId'}, inplace=True)
    data = pd.merge(df_post, df_user, on='UserId', how='left')
    data.dropna(inplace=True)
    print("Combined frame shape", data.shape)
#     data.drop(['UserId'], axis=1, inplace=True)
    data.rename(columns={'DisplayName':'Name', 'Body': 'AnswerText'}, inplace=True)
#     data = data.reindex(columns=['Name', 'AnswerText'])

    data['AnswerText'] = data['AnswerText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().replace('|', '').lower())
    data['AnswerText'] = data['AnswerText'].apply(lambda x: " ".join(x.split()))
#     soup = BeautifulSoup(data.at[0,'Post'], 'html.parser')
#     print(soup.get_text())
    data.to_csv(dataset_dir /  (category + ".csv"), sep='|', encoding='utf-8', header=False, index=False, na_rep=string)
    print("Write complete for post: ", category)
    return data

In [55]:
df = create_data('cs_stackoverflow_dataset')

Posts:  (70437, 21)
Users:  (89061, 13)
Combined frame shape (37518, 7)
Write complete for post:  cs_stackoverflow_dataset


In [56]:
df.head()

,UserId,AnswerText,PostTypeId,CreationDate,Score,Name,Reputation
0,41,in comparison to other comparison-based sortin...,2,2012-03-06T19:19:20.237,20,Kaveh,18213
1,44,"assuming you are storing a reference, not the ...",2,2012-03-06T19:44:17.600,8,Rafe Kettler,181
2,29,i think one of the main reasons why quicksort ...,2,2012-03-06T19:48:00.343,34,svick,1726
3,43,"in a truly ""cooperative"" setting, and if there...",2,2012-03-06T19:54:59.633,15,Carl Mummert,1913
5,22,even though quick-sort has a worst case run ti...,2,2012-03-06T20:18:27.510,16,0x0,565


In [57]:
df.shape

(37518, 7)

In [58]:
#UnicodeEncodeError: 'utf-8' codec can't encode character '\udbff' in position 789: surrogates not allowed

In [59]:
def cleanser(text):
    post = ''
    try:
        post = text.encode("utf-8")
    except UnicodeEncodeError as e:
        if e.reason == 'surrogates not allowed':
            post =  text.encode('utf-8', "backslashreplace").decode('utf-8')
    
    return post

In [61]:
df['AnswerText'] = df['AnswerText'].apply(lambda x: cleanser(x))

In [62]:
df.shape

(37518, 7)

In [11]:
# x= "i love this"
# print(x.encode('utf-8').decode('utf-8'))
# print(x)

i love this
i love this


In [14]:
df.head()

,Name,Post
0,Eggs McLaren,"b""i want to use a track-bar to change a form's..."
1,Kevin Dente,"b""i have an absolutely positioned div containi..."
2,Kevin Dente,"b""an explicit cast to double like this isn't n..."
3,Jeff Atwood,"b""given a datetime representing a person's bir..."
4,Jeff Atwood,"b'given a specific datetime value, how do i di..."


In [63]:
df.to_csv(dataset_dir / 'so_train_full_data.csv' , sep='|', header=False, index=False, na_rep=string)

In [72]:
def clean(text):
    post =''
    try:
        post = text.decode('utf-8')
    except AttributeError:
        post =  text
    return post

In [73]:
df['AnswerText'] = df['AnswerText'].apply(lambda x: clean(x))

In [74]:
df.head()

,UserId,AnswerText,PostTypeId,CreationDate,Score,Name,Reputation,Gender
0,41,in comparison to other comparison-based sortin...,2,2012-03-06T19:19:20.237,20,Kaveh,18213,andy
1,44,"assuming you are storing a reference, not the ...",2,2012-03-06T19:44:17.600,8,Rafe Kettler,181,unknown
2,29,i think one of the main reasons why quicksort ...,2,2012-03-06T19:48:00.343,34,svick,1726,unknown
3,43,"in a truly ""cooperative"" setting, and if there...",2,2012-03-06T19:54:59.633,15,Carl Mummert,1913,female
5,22,even though quick-sort has a worst case run ti...,2,2012-03-06T20:18:27.510,16,0x0,565,unknown


In [76]:
df.to_csv(dataset_dir / 'so_train_full_data.csv', sep='|', encoding='utf-8', header=False, index=False, na_rep=string)

In [75]:
df.shape

(37518, 8)

# gender guesser 

In [4]:
df = pd.read_csv(dataset_dir / 'so_train_full_data.csv', sep='|', encoding='utf8', quoting=csv.QUOTE_NONE)

In [77]:
df.head()

,UserId,AnswerText,PostTypeId,CreationDate,Score,Name,Reputation,Gender
0,41,in comparison to other comparison-based sortin...,2,2012-03-06T19:19:20.237,20,Kaveh,18213,andy
1,44,"assuming you are storing a reference, not the ...",2,2012-03-06T19:44:17.600,8,Rafe Kettler,181,unknown
2,29,i think one of the main reasons why quicksort ...,2,2012-03-06T19:48:00.343,34,svick,1726,unknown
3,43,"in a truly ""cooperative"" setting, and if there...",2,2012-03-06T19:54:59.633,15,Carl Mummert,1913,female
5,22,even though quick-sort has a worst case run ti...,2,2012-03-06T20:18:27.510,16,0x0,565,unknown


In [78]:
import gender_guesser.detector as gender
d = gender.Detector(case_sensitive=False)

In [79]:
translator = str.maketrans('', '', string.punctuation)

In [80]:
def get_gender(name):
    try:
        n = name.lower()
        if ('mom' in n) or ('girl' in n) or ('angel' in n) or ('mum' in n) or ('mother' in n) or ('woman' in n):
            return 'female'
        if ('boy' in n) or ('dude' in n):
            return 'male'
        temp = name.translate(translator).split()
    except:
        return 'unknown'
    
    if len(temp)>0:
        first_name = temp[0]
        first_name = ''.join([i for i in first_name if not i.isdigit()])
        gender = d.get_gender(first_name,'usa')
        if(gender=='unknown'):
            first_name= first_name[:-1]
            gender = d.get_gender(first_name,'usa')
        return gender
    else:
        return 'unknown'

In [81]:
df['Gender'] = df['Name'].apply(lambda x:get_gender(x))

In [82]:
df.Gender.value_counts()

unknown          16157
andy             12732
male              7194
mostly_male        927
mostly_female      293
female             215
Name: Gender, dtype: int64

In [71]:
df.head()

,UserId,AnswerText,PostTypeId,CreationDate,Score,Name,Reputation,Gender
0,41,"b""in comparison to other comparison-based sort...",2,2012-03-06T19:19:20.237,20,Kaveh,18213,andy
1,44,"b""assuming you are storing a reference, not th...",2,2012-03-06T19:44:17.600,8,Rafe Kettler,181,unknown
2,29,"b""i think one of the main reasons why quicksor...",2,2012-03-06T19:48:00.343,34,svick,1726,unknown
3,43,"b'in a truly ""cooperative"" setting, and if the...",2,2012-03-06T19:54:59.633,15,Carl Mummert,1913,female
5,22,"b""even though quick-sort has a worst case run ...",2,2012-03-06T20:18:27.510,16,0x0,565,unknown


In [91]:
disclosed_flag = df.Gender.isin(['mostly_male', 'male', 'mostly_female', 'female'])
undisclosed_flag = ~disclosed_flag

In [92]:
disclosed_gender_df =  df[disclosed_flag]
undisclosed_gender_df = df[undisclosed_flag]

In [86]:
df_male = df[df.Gender.isin(['mostly_male', 'male'])]

In [87]:
df_male.head()

,UserId,AnswerText,PostTypeId,CreationDate,Score,Name,Reputation,Gender
15,69,"in relational algebra, we shall first provide ...",2,2012-03-06T21:59:27.307,14,Patrick87,10544,male
16,71,"recall that a pair $(a[i], a[j])$ (resp. $(i,j...",2,2012-03-06T22:04:37.040,19,Joe,3205,male
18,92,the context-sensitive languages are exactly th...,2,2012-03-06T22:39:46.867,19,Alex ten Brink,7181,mostly_male
20,98,modern programming language use context-sensit...,2,2012-03-06T22:51:25.753,4,Raphael,58925,male
21,95,"in context-free languages, at any point of the...",2,2012-03-06T22:55:34.103,8,Victor Stafusa,957,male


In [88]:
df_female=df[df.Gender.isin(['mostly_female', 'female'])]

In [89]:
df_female.head()

,UserId,AnswerText,PostTypeId,CreationDate,Score,Name,Reputation,Gender
3,43,"in a truly ""cooperative"" setting, and if there...",2,2012-03-06T19:54:59.633,15,Carl Mummert,1913,female
8,43,one direct way is a recursive procedure that d...,2,2012-03-06T20:33:49.193,14,Carl Mummert,1913,female
23,43,a general theorem that partially covers the ex...,2,2012-03-06T23:33:49.507,14,Carl Mummert,1913,female
59,43,"yes, this is decidable, because you can do an ...",2,2012-03-09T01:03:08.753,8,Carl Mummert,1913,female
125,43,because of diagonalization. if $(f_e: e \in \m...,2,2012-03-13T01:26:15.257,24,Carl Mummert,1913,female


In [93]:
import os.path
import pandas as pd
if not os.path.exists(abs_data_dir / '..' / 'disclosed_dataset.csv'):
    print('saving disclosed dataset to csv')
    disclosed_gender_df.to_csv(abs_data_dir / '..' / 'disclosed_dataset.csv', index=False)
else:
    disclosed_gender_df = pd.read_csv(abs_data_dir / '..' / 'disclosed_dataset.csv')

if not os.path.exists(abs_data_dir / '..' / 'undisclosed_dataset.csv'):
    print('saving undisclosed dataset to csv')
    undisclosed_gender_df.to_csv(abs_data_dir / '..' / 'undisclosed_dataset.csv', index=False)
else:
    undisclosed_gender_df = pd.read_csv(abs_data_dir / '..' / 'undisclosed_dataset.csv')

saving disclosed dataset to csv
saving undisclosed dataset to csv


In [96]:
dataset_dir = Path.cwd() / 'csv_files/stackexchange/'

In [101]:
disclosed_gender_df.rename(columns={'Name': 'UserName', 'AnswerText': 'Text', 'CreationDate': 'Timestamp'}, inplace=True)
disclosed_gender_df.head()

,UserId,Text,PostTypeId,Timestamp,Score,UserName,Reputation,Gender
3,43,"in a truly ""cooperative"" setting, and if there...",2,2012-03-06T19:54:59.633,15,Carl Mummert,1913,female
8,43,one direct way is a recursive procedure that d...,2,2012-03-06T20:33:49.193,14,Carl Mummert,1913,female
15,69,"in relational algebra, we shall first provide ...",2,2012-03-06T21:59:27.307,14,Patrick87,10544,male
16,71,"recall that a pair $(a[i], a[j])$ (resp. $(i,j...",2,2012-03-06T22:04:37.040,19,Joe,3205,male
18,92,the context-sensitive languages are exactly th...,2,2012-03-06T22:39:46.867,19,Alex ten Brink,7181,mostly_male


In [102]:
undisclosed_gender_df.rename(columns={'Name': 'UserName', 'AnswerText': 'Text', 'CreationDate': 'Timestamp'}, inplace=True)
undisclosed_gender_df.head()

,UserId,Text,PostTypeId,Timestamp,Score,UserName,Reputation,Gender
0,41,in comparison to other comparison-based sortin...,2,2012-03-06T19:19:20.237,20,Kaveh,18213,andy
1,44,"assuming you are storing a reference, not the ...",2,2012-03-06T19:44:17.600,8,Rafe Kettler,181,unknown
2,29,i think one of the main reasons why quicksort ...,2,2012-03-06T19:48:00.343,34,svick,1726,unknown
5,22,even though quick-sort has a worst case run ti...,2,2012-03-06T20:18:27.510,16,0x0,565,unknown
6,59,in the cooperative scheduling (preferably coop...,2,2012-03-06T20:19:10.463,4,Ankit,627,andy


In [112]:
mapping = {'female' : 1, 'male' : 0, 'mostly_female': 1, 'mostly_male': 0}
disclosed_dataset_df = disclosed_gender_df[['UserName', 'Text', 'Gender']]
disclosed_dataset_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
disclosed_dataset_df.replace({'Gender': mapping}, inplace=True)

undisclosed_gender_df = undisclosed_gender_df[['UserName', 'Text', 'Gender']]
undisclosed_gender_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
undisclosed_gender_df.to_csv(dataset_dir / 'undisclosed_id_text_gender.csv', index=False, header=False)

/home/yafei/.conda/envs/ma/lib/python3.7/site-packages/pandas/core/generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [113]:
from sklearn.model_selection import train_test_split

# dataset_df = pd.read_csv('train_dataset.csv')
if 'disclosed_dataset_df' in locals():
    # 80/20 train/test
    train_df, test_df = train_test_split(disclosed_dataset_df, test_size=0.2)
    # 80/20 train/validation
    train_df, validation_df = train_test_split(train_df, test_size=0.2)

In [114]:

# extracting related attributes for training, validation and test
train_gender_text_df = train_df[['Gender', 'Text']]
validation_gender_text_df = validation_df[['Gender', 'Text']]
test_name_text_gender_df = test_df[['UserName', 'Text', 'Gender']]

train_gender_text_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
train_gender_text_df.to_csv(dataset_dir / 'training_gender_text.csv', index=False, header=False)

validation_gender_text_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
validation_gender_text_df.to_csv(dataset_dir / 'validation_gender_text.csv', index=False, header=False)

test_name_text_gender_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
test_name_text_gender_df.to_csv(dataset_dir / 'test_name_text_gender.csv', index=False, header=False)

In [108]:
import pandas as pd
undisclosed_csv_df = pd.read_csv(dataset_dir / 'undisclosed_dataset.csv')
undisclosed_csv_df.head()

,UserId,AnswerText,PostTypeId,CreationDate,Score,Name,Reputation,Gender
0,41,in comparison to other comparison-based sortin...,2,2012-03-06T19:19:20.237,20,Kaveh,18213,andy
1,44,"assuming you are storing a reference, not the ...",2,2012-03-06T19:44:17.600,8,Rafe Kettler,181,unknown
2,29,i think one of the main reasons why quicksort ...,2,2012-03-06T19:48:00.343,34,svick,1726,unknown
3,22,even though quick-sort has a worst case run ti...,2,2012-03-06T20:18:27.510,16,0x0,565,unknown
4,59,in the cooperative scheduling (preferably coop...,2,2012-03-06T20:19:10.463,4,Ankit,627,andy


In [116]:
undisclosed_dataset_df = undisclosed_csv_df[['UserId', 'AnswerText', 'Gender']]
# undisclosed_dataset_df.rename(columns={'Name': 'UserName', 'AnswerText': 'Text', 'CreationDate': 'Timestamp'}, inplace=True)
undisclosed_dataset_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
undisclosed_dataset_df.to_csv(dataset_dir / 'undisclosed_id_text_gender.csv', index=False, header=False)

In [115]:
undisclosed_dataset_df.head()

,UserId,Text,Gender
0,41,in comparison to other comparison-based sortin...,andy
1,44,"assuming you are storing a reference, not the ...",unknown
2,29,i think one of the main reasons why quicksort ...,unknown
3,22,even though quick-sort has a worst case run ti...,unknown
4,59,in the cooperative scheduling (preferably coop...,andy


# Additional Info

In [38]:
df2 = data.groupby('Name').agg({'Post':'count','Gender':'first'}).reset_index().rename(columns={'Post':'Count'})

In [39]:
df2.head()

,Name,Count,Gender
0,Maria,1,female
1,Niki Hard,1,female
2,Allen,1,male
3,Anna,2,female
4,Benny Duikelaar,1,male


In [41]:
df_m = df2.loc[df2.Gender=='male']
df_f = df2.loc[df2.Gender=='female']
print(df_m.shape,df_f.shape)

(107654, 3) (54061, 3)


In [42]:
train_ids =[]
test_ids=[]
count=0
c_tr = 0
c_ts=0
limit = round(.75*df_m.Count.sum())
for index, row in df_m.iterrows():
    count+= row['Count']
    
    if(count<= limit):
        train_ids.append(row['Name'])
        c_tr+= row['Count']
    else:
        test_ids.append(row['Name'])
        c_ts+= row['Count']

print(len(train_ids),c_tr)
print(len(test_ids),c_ts)

79871 461248
27783 154740


In [43]:
count=0
c_tr = 0
c_ts=0
limit = round(.75*df_f.Count.sum())
for index, row in df_f.iterrows():
    count+= row['Count']
    
    if(count<= limit):
        train_ids.append(row['Name'])
        c_tr+= row['Count']
    else:
        test_ids.append(row['Name'])
        c_ts+= row['Count']

print(len(train_ids),c_tr)
print(len(test_ids),c_ts)

121888 462017
39827 154040


In [44]:
df_train = data.loc[data.Name.isin(train_ids)].reset_index()

In [45]:
df_train.head()

,index,Name,Post,Gender
0,0,Joseph Mansfield,"""that's what the carriage return character is ...",male
1,1,Hank,since i have no control of which connections a...,male
2,2,Quentin Hayot,if you really need to download your data every...,male
3,3,James Dunay,"""i can't figure out whats happening here and w...",male
4,5,Brian Ball,there are already some good suggestions. i wou...,male


In [47]:
df_train.shape

(923341, 4)

In [48]:
df_test = data.loc[data.Name.isin(test_ids)].reset_index()

In [49]:
df_test.shape

(308789, 4)

In [50]:
df_train.drop(['index'],axis=1, inplace=True)
df_test.drop(['index'],axis=1, inplace=True)

In [51]:
df_train.drop(['Name'],axis=1, inplace=True)

In [52]:
df_train['Label'] = pd.get_dummies(df_train.Gender)['female']
df_train.drop(['Gender'],axis=1, inplace=True)
df_train.head()

,Post,Label
0,"""that's what the carriage return character is ...",0
1,since i have no control of which connections a...,0
2,if you really need to download your data every...,0
3,"""i can't figure out whats happening here and w...",0
4,there are already some good suggestions. i wou...,0


In [53]:
df_train = df_train[['Label','Post']]

In [54]:
df_train.head()

,Label,Post
0,0,"""that's what the carriage return character is ..."
1,0,since i have no control of which connections a...
2,0,if you really need to download your data every...
3,0,"""i can't figure out whats happening here and w..."
4,0,there are already some good suggestions. i wou...


In [55]:
df_train = df_train.sample(frac = 1).reset_index(drop=True)
df_train.head()

,Label,Post
0,0,you should store the fact that that a row as b...
1,0,django 1.9.7 postgresql could you help me unde...
2,1,"""we have a lot of ajax calls in our applicatio..."
3,0,"""i am trying to implement a program that will ..."
4,1,"when an element is fixed, it needs its coordin..."


In [56]:
df_train.to_csv("/home/rachneet/datasets/stack_overflow/train_so.csv",encoding = 'utf-8',header=False,index=False,na_rep=string)

In [57]:
df_test['Label'] = pd.get_dummies(df_test.Gender)['female']
df_test.drop(['Gender'],axis=1, inplace=True)
df_test = df_test[['Name','Label','Post']]
df_test = df_test.sample(frac = 1).reset_index(drop=True)
df_test.head()

,Name,Label,Post
0,Zack Macomber,0,"""i have the following scope function in my con..."
1,Tim,0,"file.readlines returns an ienumerable<string>,..."
2,Xander Angell,1,i am working in html 5 and wanted to add an im...
3,Trevor Hickey,0,"""i understand the motivation for using std::st..."
4,murielg,1,"""i am trying to append a string, myoutput, rig..."


In [58]:
x = round(df_test.shape[0] * 0.80)
test_set = df_test.iloc[:x]
val_set = df_test.iloc[x:]
print(test_set.shape, val_set.shape)

(247031, 3) (61758, 3)


In [59]:
test_set.head()

,Name,Label,Post
0,Zack Macomber,0,"""i have the following scope function in my con..."
1,Tim,0,"file.readlines returns an ienumerable<string>,..."
2,Xander Angell,1,i am working in html 5 and wanted to add an im...
3,Trevor Hickey,0,"""i understand the motivation for using std::st..."
4,murielg,1,"""i am trying to append a string, myoutput, rig..."


In [60]:
test_set.to_csv("/home/rachneet/datasets/stack_overflow/test_so.csv",encoding = 'utf-8',header=False,index=False,na_rep=string)

In [61]:
val_set.drop(['Name'],axis=1, inplace=True)
val_set.head()

/home/rachneet/.local/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Label,Post
247031,0,"""i did it shown in this example: http://blog.s..."
247032,0,"""please read the comments to this post underst..."
247033,0,"""i'm trying to hide/show an action bar button ..."
247034,1,i'm wondering where are gcc macros like builti...
247035,1,i am looking for a good wiki software to host ...


In [62]:
val_set.to_csv("/home/rachneet/datasets/stack_overflow/validation_so.csv",encoding = 'utf-8',header=False,index=False,na_rep=string)